# Colab setting

In [11]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: './drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book'
/content/drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book


In [12]:
%ls

bench_mark.ipynb  Cat_colab.ipynb             learn_categorical_plots.ipynb
catboost_info/    colab_lgbt_multigrid.ipynb  Light_GBM.ipynb
Catboost.ipynb    edm_data.ipynb


Install lgbm with gpu option

ref: https://an-engineer-note.com/?p=624

In [13]:
!pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

Found existing installation: lightgbm 3.3.5
Uninstalling lightgbm-3.3.5:
  Successfully uninstalled lightgbm-3.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lightgbm-3.3.5.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
Skipping wheel build for lightgbm, due to binaries being disabled for it.
  Running setup.py install for lightgbm ... done


# lgbm

In [14]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
import pandas as pd
import warnings
import pprint

import lightgbm as lgb
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from src import common

## grid search

In [16]:
train_values, train_labels = common.file.read_data('train')

In [17]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [18]:
features_list = ['geo_level_1_id', 
                 'geo_level_2_id', 
                 'geo_level_3_id',
                 'age', 
                 'area_percentage', 
                 'height_percentage', 
                 'foundation_type', 
                 'roof_type', 
                 'ground_floor_type', 
                 'other_floor_type', 
                 'position', 
                 'has_superstructure_mud_mortar_stone', 
                 'has_superstructure_cement_mortar_brick', 
                 'has_superstructure_timber', 
                 'count_families', 
                 'has_secondary_use'
                ]

In [19]:
train_values, train_labels = common.lgbm_preprocessing((train_values, train_labels), mode='training', features_list=features_list)

ValueError: ignored

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_values, train_labels,
                                                        test_size=0.1, random_state=19, stratify=train_labels)

In [ ]:
skf = StratifiedKFold(n_splits=5,
                      shuffle=True,
                      random_state=0)

In [ ]:
# initialize model and parameter grid
model = lgb.LGBMClassifier(objective='multi_class', num_class=3, 
                           learning_rate=0.15, 
                           force_row_wise=True, 
                           num_boost_round=200, 
                           device='gpu', 
                           # valid_sets=valids
                           )

param_grid = {
    "max_depth": [10, 25, 50, 75],
    # "learning_rate" : [0.001,0.01,0.05,0.1],
    # "num_leaves": [100,300,900,1200],
    "n_estimators": [100,200,500], 
    # 'reg_alpha': list(map(lambda x: x*0.1, range(1, 5))), 
    # 'reg_lambda': [0.1, 0.2],
             }

# grid search
grid_result = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           scoring = 'f1_micro', 
                           cv = skf,
                           return_train_score = True,
                           n_jobs = -1)

grid_result.fit(x_train, np.reshape(y_train, -1))

In [ ]:
pprint.pprint(grid_result.best_estimator_)

In [ ]:
pprint.pprint(grid_result.best_score_)

# Use best estimator 

In [ ]:
train_values, train_labels = common.file.read_data('train')

In [ ]:
features_list = ['geo_level_1_id', 
                 'geo_level_2_id', 
                 'geo_level_3_id',
                 'age', 
                 'area_percentage', 
                 'height_percentage', 
                 'foundation_type', 
                 'roof_type', 
                 'ground_floor_type', 
                 'other_floor_type', 
                 'position', 
                 'has_superstructure_mud_mortar_stone', 
                 'has_superstructure_cement_mortar_brick', 
                 'has_superstructure_timber', 
                 'count_families', 
                 'has_secondary_use'
                ]

In [ ]:
train_values, train_labels = common.lgbm_preprocessing((train_values, train_labels), mode='training', features_list=features_list)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_values, train_labels,
                                                        test_size=0.1, random_state=19, stratify=train_labels)

In [ ]:
trains = lgb.Dataset(x_train, y_train)
valids = lgb.Dataset(x_test, y_test)

In [ ]:
params = {
    "objective": "multiclass",
    "num_class": 3, 
    "metrics": "multi_logloss",
    'device': 'gpu', 
    'max_depth': 75, 
    'force_row_wise': True, 
    "learning_rate": 0.05, 
    'reg_alpha': 0.3,
    'reg_lambda': 0.3,
}

In [ ]:
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=20000, early_stopping_rounds=1000)

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


Streaming output truncated to the last 5000 lines.
[86]	valid_0's multi_logloss: 0.665413
[87]	valid_0's multi_logloss: 0.665027
[88]	valid_0's multi_logloss: 0.664725
[89]	valid_0's multi_logloss: 0.664346
[90]	valid_0's multi_logloss: 0.663815
[91]	valid_0's multi_logloss: 0.663457
[92]	valid_0's multi_logloss: 0.66309
[93]	valid_0's multi_logloss: 0.662702
[94]	valid_0's multi_logloss: 0.662283
[95]	valid_0's multi_logloss: 0.661894
[96]	valid_0's multi_logloss: 0.661608
[97]	valid_0's multi_logloss: 0.661283
[98]	valid_0's multi_logloss: 0.660882
[99]	valid_0's multi_logloss: 0.660517
[100]	valid_0's multi_logloss: 0.660236
[101]	valid_0's multi_logloss: 0.659953
[102]	valid_0's multi_logloss: 0.659691
[103]	valid_0's multi_logloss: 0.659399
[104]	valid_0's multi_logloss: 0.659159
[105]	valid_0's multi_logloss: 0.658818
[106]	valid_0's multi_logloss: 0.658576
[107]	valid_0's multi_logloss: 0.658179
[108]	valid_0's multi_logloss: 0.657758
[109]	valid_0's multi_logloss: 0.65748
[110]

In [ ]:
# predict

# テストデータの予測 ((各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す))
y_pred_prob = model.predict(x_test)
# テストデータの予測 (予測クラス(0 or 1 or 2)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
# 真値と予測値の表示
df_pred = pd.DataFrame({'target':y_test['damage_grade'].values,'target_pred':y_pred})
display(df_pred)

# 真値と予測確率の表示
df_pred_prob = pd.DataFrame({'y':y_test['damage_grade'].values, 'target0_prob':y_pred_prob[:,0], 'target1_prob':y_pred_prob[:,1], 'target2_prob':y_pred_prob[:,2]})
display(df_pred_prob)

acc = f1_score(y_test,y_pred, average='micro')
print('Acc :', acc)

,target,target_pred
0,1,1
1,2,2
2,1,2
3,1,1
4,1,1
...,...,...
26056,0,1
26057,1,1
26058,1,1
26059,0,1


,y,target0_prob,target1_prob,target2_prob
0,1,0.042023,0.622313,0.335664
1,2,0.002117,0.454850,0.543033
2,1,0.000342,0.399571,0.600088
3,1,0.001897,0.894754,0.103348
4,1,0.220040,0.709849,0.070111
...,...,...,...,...
26056,0,0.452068,0.494931,0.053000
26057,1,0.201995,0.573719,0.224286
26058,1,0.349016,0.605332,0.045652
26059,0,0.257338,0.699821,0.042841


Acc : 0.7465177851962704


# submit

In [ ]:
test_values = common.file.read_data('test')
test_values, _ = common.lgbm_preprocessing(test_values, mode='test', features_list=features_list)
y_test_prob = model.predict(test_values)
y_test = np.argmax(y_test_prob, axis=1)
submission_format = common.file.read_data('submission')
my_submission = pd.DataFrame(data=y_test+1,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('../../../data/final/submit/gridsearch_20000trial.csv')

# Custom metric

In [20]:
from sklearn.metrics import f1_score

def f1(y_true, y_pred):
    N_LABELS = 3  # ラベルの数
    y_pred_ = y_pred.reshape(N_LABELS, len(y_pred) // N_LABELS).argmax(axis=0)
    score = f1_score(y_true, y_pred_, average='micro')
    return 'f1', score, True

In [21]:
train_values, train_labels = common.file.read_data('train')

features_list = ['geo_level_1_id', 
                 'geo_level_2_id', 
                 'geo_level_3_id',
                 'age', 
                 'area_percentage', 
                 'height_percentage', 
                 'foundation_type', 
                 'roof_type', 
                 'ground_floor_type', 
                 'other_floor_type', 
                 'position', 
                 'has_superstructure_mud_mortar_stone', 
                 'has_superstructure_cement_mortar_brick', 
                 'has_superstructure_timber', 
                 'count_families', 
                 'has_secondary_use'
                ]


train_values, train_labals = common.lgbm_preprocessing((train_values, train_labels), features_list=features_list)

In [22]:
# split to train and test

X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, 
                                                    test_size=0.1, random_state=19, stratify=train_labels)

trains = lgb.Dataset(X_train, y_train)
valids = lgb.Dataset(X_test, y_test)

params = {
    "objective": "multiclass",
    "num_class": 3, 
    'device': 'gpu', 
    'force_row_wise': True, 
    "learning_rate": 0.05, 
    'reg_alpha': 0.3,
    'reg_lambda': 0.3,
    'metric': None, 
    'n_estimators': 20000, 
}

In [23]:
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train, 
          eval_set=[(X_test, y_test)], 
          early_stopping_rounds=1000, 
          eval_metric=f1)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Re

[1]	valid_0's multi_logloss: 0.89156	valid_0's f1: 0.568896
[2]	valid_0's multi_logloss: 0.873445	valid_0's f1: 0.568896
[3]	valid_0's multi_logloss: 0.857587	valid_0's f1: 0.568896
[4]	valid_0's multi_logloss: 0.843603	valid_0's f1: 0.568896
[5]	valid_0's multi_logloss: 0.831116	valid_0's f1: 0.627029
[6]	valid_0's multi_logloss: 0.820018	valid_0's f1: 0.631633
[7]	valid_0's multi_logloss: 0.810093	valid_0's f1: 0.632132
[8]	valid_0's multi_logloss: 0.801028	valid_0's f1: 0.633437
[9]	valid_0's multi_logloss: 0.792736	valid_0's f1: 0.634396
[10]	valid_0's multi_logloss: 0.785103	valid_0's f1: 0.639269
[11]	valid_0's multi_logloss: 0.778152	valid_0's f1: 0.641188
[12]	valid_0's multi_logloss: 0.772035	valid_0's f1: 0.646982
[13]	valid_0's multi_logloss: 0.766179	valid_0's f1: 0.648479
[14]	valid_0's multi_logloss: 0.760717	valid_0's f1: 0.651433
[15]	valid_0's multi_logloss: 0.755581	valid_0's f1: 0.65692
[16]	valid_0's multi_logloss: 0.750902	valid_0's f1: 0.665976
[17]	valid_0's mult

LGBMClassifier(device='gpu', force_row_wise=True, learning_rate=0.05,
               metric=None, n_estimators=20000, num_class=3,
               objective='multiclass', reg_alpha=0.3, reg_lambda=0.3)

In [25]:
# テストデータの予測 ((各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率,クラス2の予測確率] を返す))
y_pred = model.predict(X_test)

acc = f1_score(y_test,y_pred, average='micro')
print('Acc :', acc)

Acc : 0.7474003299950117


In [26]:
test_values = common.file.read_data('test')
test_values, _ = common.lgbm_preprocessing(test_values, mode='test', features_list=features_list)
y_test = model.predict(test_values)
submission_format = common.file.read_data('submission')
my_submission = pd.DataFrame(data=y_test+1,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('../../../data/final/submit/lgbm_custommetric_20000trial.csv')

,damage_grade
building_id,
300051,4
99355,3
890251,3
745817,2
421793,4
...,...
310028,3
663567,3
1049160,3
